In [1]:
import pandas as pd
import itertools
import cobra
from typing import Dict, List
from collections import defaultdict

E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
gene_pathway_df = pd.read_csv('./Data/gene_pathway.csv')
desired_cycle = pd.read_csv('./Data/desired_cycle.csv', index_col=0) #flux analysis.ipynb
gr_DG = pd.read_csv('./r/data fitting/standardized_gr_Div_DG_Mult_Aug31.csv', index_col='gene_inhibition')


In [2]:
def get_col_p_o(gr_df):
    additive_threshold = .01
    diff_bins = [-10,-1*additive_threshold,1*additive_threshold,10]
    
    if gr_df.index.name == 'gene_inhibition':
        gr_df.index.name = 'Gene_inhibition'
    col_p_o = gr_df.Predicted_additive_effect_E0_coculture-gr_df.E0_coculture
    col_p_o[(gr_df.Predicted_additive_effect_E0_coculture < 1.5e-8) & (gr_df.E0_coculture< 1.5e-8)] = 0
    df = pd.DataFrame(col_p_o, columns=['P_O'])
    df['gene_sort'] = (df['P_O'] < 0)*10 + abs(df['P_O'])
    
    df['Drug_comb_effect'] = pd.cut(df['P_O'], bins=diff_bins, labels=['Antagonistic', 'Additive', 'Synergistic'])
    return df

def get_antagonistic_df(syn_df):
    antagonistic_list = syn_df.loc[syn_df['P_O']<0].query("P_O<-0.01").index 
    # ? func get pwy col
    potential_pwy = (gene_combo_pathway
                                      .loc[antagonistic_list,'Pathway']
                                      .str.split(' \+ ') # series string need \+
                                      .apply(lambda x: sorted(x))) 
    return pd.DataFrame(potential_pwy)

In [90]:
def add_single_pwy_to_p_o(p_o):
    rows = (syn_df.loc[DG_single_pathway_df.Gene_inhibition]
            .reset_index()
            .merge(DG_single_pathway_df[['Gene_inhibition', 'First_Pathway', 'Pathway_count']]))
    return pd.concat([rows, p_o])
    
def get_p_o_df(potential_pwy_list):
    df_list = list()
    # !? sep get index -> get antagon index directly
    for desired_pathway_list in potential_pwy_list:
        # cater for pathway > 2 -> nC2 pairwise combination
        for pathways in itertools.combinations(desired_pathway_list,2): #2C2 if 2 pathway supplied in tuple
            temp_df = gene_pathway_df.query(' Pathway in @pathways & "DG" in XG').groupby('Gene_inhibition').size()>=2
            temp_df = (syn_df.loc[temp_df[temp_df == True].index] # filter gcomb occur in both pathway 
    #                    .sort_values('gene_sort', ascending=False)
    #                    .drop('Sort', axis=1))
                      )
            if len(temp_df.index)>=1:
                diff_noise = np.random.uniform(0,.01)
                temp_df['group_sort'] = max(temp_df['gene_sort'])+diff_noise
                temp_df['First_Pathway'] = pathways[0]
                temp_df['Second_Pathway'] = pathways[1]
                df_list.append(temp_df)
    p_o = (pd.concat(df_list)
           .sort_values('gene_sort',ascending=False)
           .reset_index()
           .merge(single_pathway_df[['Gene_inhibition', 'Pathway_count']]))
    p_o = add_single_pwy_to_p_o(p_o)
    return p_o.set_index('Gene_inhibition')

# read from single pathway->determine first & second

In [55]:
single_pathway_df = pd.read_csv('./Data/single_pathway_df.csv')
DG_single_pathway_df = single_pathway_df.query('XG=="DG" & Pathway_count ==1').copy()
DG_single_pathway_df['First_Pathway'] = DG_single_pathway_df.Pathway_list.apply(lambda x: ast.literal_eval(x)[0])

In [58]:
syn_df.loc[DG_single_pathway_df.Gene_inhibition].reset_index().merge(DG_single_pathway_df[['Gene_inhibition', 'First_Pathway']])

,Gene_inhibition,P_O,gene_sort,Drug_comb_effect,First_Pathway
0,aroA.pheA,0.002529,0.002529,Additive,"Tyrosine, Tryptophan, and Phenylalanine Metabo..."
1,dapA.dapF,2.032069,2.032069,Synergistic,Threonine and Lysine Metabolism
2,dapF.dapD,0.250342,0.250342,Synergistic,Threonine and Lysine Metabolism
3,folC.folA,0.032817,0.032817,Synergistic,Cofactor and Prosthetic Group Biosynthesis
4,folP.folA,0.021941,0.021941,Synergistic,Cofactor and Prosthetic Group Biosynthesis
5,folP.folC,0.016356,0.016356,Synergistic,Cofactor and Prosthetic Group Biosynthesis
6,mrdA.mrcA,0.294345,0.294345,Synergistic,Murein Biosynthesis
7,pyrE.pyrD,0.605203,0.605203,Synergistic,Purine and Pyrimidine Biosynthesis
8,thrC.thrB,0.144768,0.144768,Synergistic,Threonine and Lysine Metabolism
9,tktA.gnd,-0.000775,10.000775,Additive,Pentose Phosphate Pathway


In [91]:
syn_df = get_col_p_o(gr_DG)
pathway_pair_list = list(itertools.combinations(sorted(gene_pathway_df.Pathway.unique()),2))
p_o = get_p_o_df(pathway_pair_list)
p_o.to_csv('./Data/pair_pathway_drug_effect.csv')

In [92]:
def update_single_pathway_df():
    df = pd.read_csv('./Data/single_pathway_df.csv')
    if 'P_O' not in df.columns:
        df.merge(p_o.reset_index()[['Gene_inhibition', 'P_O']],
                how='left').to_csv('./Data/single_pathway_df.csv', index=False)
update_single_pathway_df()